In [1]:
import numpy as np
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("NormalizacionDataset").getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/18 19:03:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
def readFile(filename:str):
    rdd = sc.textFile(filename)
    return (
        rdd.map(lambda x: x.split(","))
        .map(lambda x: tuple(map(float, x)))
        .map(lambda x: (x[:-1], int(x[-1])))
    )

In [ ]:
rdd = readFile("data/botnet_reduced_10k_l.csv")
TOTAL = rdd.count()
rdd.take(2)

[((3545.3018916840147,
   3198.0139469522546,
   80.00015469454229,
   1.0000019205086303,
   444960086.4643011,
   476.8073064086366,
   12.999999992936175,
   -7.710424743123667e-09,
   87.00000148917339,
   19099430.15956688,
   2468368394.7593513),
  0),
 ((-1.21308761436012e-06,
   7.999989079213265,
   65499.99887793755,
   0.9999997452701503,
   61.99988768910407,
   69.99980788311223,
   11.999999995904211,
   -2.35688187855132e-08,
   8.00000498340475,
   2468369617.006896,
   2468368392.7377276),
  1)]

In [4]:
def normalize(rdd_xy):
    rdd_X = rdd_xy.map(lambda x: x[0])
    _ = rdd_xy.map(lambda x: x[1])

    mu, rows = rdd_X.map(lambda x: (np.array(x), 1)).reduce(lambda x, y: (x[0] + y[0], x[1] + y[1]))
    mu /= rows
    std = (rdd_X.map(lambda x: (np.array(x) - mu) ** 2).reduce(lambda x, y: x + y) / rows) ** 0.5

    return rdd_xy.map(lambda x: ((np.array(x[0]) - mu) / std, x[1]))

In [ ]:
normalize(rdd).take(2)

In [ ]:
def train(rdd_xy, iterations, learning_rate, lambda_reg):
    its_to_print = list(range(0, iterations, iterations // 10)) + [iterations]
    k = len(rdd_xy.first()[0])
    m = rdd_xy.count()
    w = np.random.rand(k)
    b = 0

    for it in range(iterations):
        # Gradients
        dw =  rdd_xy.map(lambda x: (predict_proba(w, b, x[0]) - x[1])*np.array(x[0])).reduce(lambda x, y: x + y) / m
        db = rdd_xy.map(lambda x: predict_proba(w, b, x[0]) - x[1]).reduce(lambda x, y: x + y) / m

        # Regularization
        dw += lambda_reg / k * w

        # Update parameters
        w -= learning_rate * dw
        b -= learning_rate * db
        
        # Logging
        if it in its_to_print:
            c = cost(w, b, rdd_xy, lambda_reg, k, m)
            acc = accuracy(w, b, rdd_xy)
            print(f"Iteration {it}/{iterations}, Cost: {c:.4f}, Accuracy: {acc:.4f}")

    return w, b

def sigmoid(logit):
    return 1 / (1 + np.exp(-logit))

def predict_proba(w, b, X):
    X = np.array(X)
    w = np.array(w)
    return sigmoid(np.dot(w, X) + b)

def predict(w, b, X):
    return 1 if predict_proba(w, b, X) > 0.5 else 0

def accuracy(w, b, rdd_xy):
    preds = rdd_xy.map(lambda x: (int(predict(w, b, x[0]) == x[1]), 1)).reduce(lambda x, y: (x[0] + y[0], x[1] + y[1]))
    return preds[0] / preds[1]

def cost(w, b, rdd_xy, lambda_reg, k, m):
    return rdd_xy.map(lambda x: (predict_proba(w, b, x[0]), x[1])).map(lambda x: (x[1]*np.log(x[0]) + (1-x[1])*np.log(1-x[0]))).reduce(lambda x, y: x + y)/(-m)+lambda_reg*(w**2).sum()/2*k

In [7]:
path = "data/botnet_reduced_10k_l.csv"  # botnet_tot_syn_l.csv // botnet_reduced_10k_l.csv
nIter = 20
leaning_rate =  1.5
lambda_reg = 0

# Read data
data = readFile(path).cache()
print(f"Total samples: {data.count()}")

# Standardize
data = normalize(data)
print("Data normalized.")
# TODO: Demonstrate normalization by showing mean and std before and after normalization

w_b = train(data, nIter, leaning_rate, lambda_reg)
acc = accuracy(w_b[0], w_b[1], data)
print(f"Accuracy: {acc:.4f}")

Total samples: 10000
Data normalized.
Iteration 0/20, Cost: 0.7286, Accuracy: 0.5855
Iteration 2/20, Cost: 0.3531, Accuracy: 0.8689
Iteration 4/20, Cost: 0.2707, Accuracy: 0.9126
Iteration 6/20, Cost: 0.2388, Accuracy: 0.9228
Iteration 8/20, Cost: 0.2220, Accuracy: 0.9272
Iteration 10/20, Cost: 0.2115, Accuracy: 0.9300
Iteration 12/20, Cost: 0.2042, Accuracy: 0.9317
Iteration 14/20, Cost: 0.1989, Accuracy: 0.9329
Iteration 16/20, Cost: 0.1948, Accuracy: 0.9337
Iteration 18/20, Cost: 0.1915, Accuracy: 0.9333
Accuracy: 0.9333
